In [1]:
import autograd.numpy as np

import wecopttool as wot

In [2]:
file = "./bem.nc"
data = wot.read_netcdf(file)
f1 = 0.05
nfreq = 50

In [3]:
amplitude = 0.0625  
idx = 6
freq = idx*f1
waves = wot.waves.regular_wave(f1, nfreq, freq, amplitude)

In [4]:
def pto_velocity(wec, x_wec, x_opt, waves):
    pos = wec.vec_to_dofmat(x_wec)
    vel = np.dot(wec.derivative_mat, pos)
    return np.dot(wec.time_mat, vel)

def objective_fun(wec, x_wec, x_opt, waves):
    force_td = pto_force(wec, x_wec, x_opt, None)
    vel_td = pto_velocity(wec, x_wec, x_opt, None)
    power_td = vel_td * force_td
    return np.sum(power_td) * wec.dt / wec.tf

def pto_force(wec, x_wec, x_opt, waves):
    return np.dot(wec.time_mat[:, 1:], wec.vec_to_dofmat(x_opt))

nstate_opt = 2 * nfreq

In [5]:
data = wot.add_zerofreq_to_xr(data)
wec = wot.WEC.from_bem(data, f_add={'PTO': pto_force})

Linear damping for DOF "HEAVE" has negative or close to zero terms. Shifting up via linear friction of 36.667232716402985 N/(m/s).


In [6]:
res = wec.solve(waves, objective_fun, nstate_opt)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -135.07335929241142
            Iterations: 42
            Function evaluations: 42
            Gradient evaluations: 42


In [16]:
w = data.coords['omega'][idx]
Z = ((1j*w)*(data['mass'] + data['added_mass'][0,0,idx])
     + data['radiation_damping'][0,0,idx]
     + (1/(1j*w)) * data['hydrostatic_stiffness']
     + 36.667232716402985
     )

exc_coeff = data['Froude_Krylov_force'] + data['diffraction_force']
F = wot.wave_excitation(exc_coeff, waves)[idx]

power_theoretical = (np.abs(F)**2 / (8 * Z.real)).values[0,0]
power = -res.fun

power, power_theoretical

(135.07335929241142, 135.0755469893718)